In [1]:
import h5py
import os
import nibabel as nib
import torch
import numpy as np

datapath = "/home/lm/Schreibtisch/Uni/MedicalML/pytorch-3dunet/data/training"

files = os.listdir(datapath)
files = filter(lambda x: x.endswith("_orig.nii.gz"), files)
files = list(map(lambda x: x.replace("_orig.nii.gz", ""), files))
files

FileNotFoundError: [Errno 2] No such file or directory: '/home/lm/Schreibtisch/Uni/MedicalML/pytorch-3dunet/data/training'

In [14]:
image_raw = nib.load(f"/home/philipp/uni/Med/data/training/A001_orig.nii.gz")

In [15]:
print(image_raw.header.get("pixdim"))


[1.       0.27148  0.27148  0.270863 0.       0.       0.       0.      ]


In [2]:
images = []
labels = []
affines = []
cases = []

for case in files:
        image_raw = nib.load(f"{datapath}/{case}_orig.nii.gz")
        mask_raw = nib.load(f"{datapath}/{case}_masks.nii.gz")

        image = image_raw.get_fdata()
        mask = mask_raw.get_fdata()

        if image.shape != mask.shape:
                print("Dont match")
                continue

        if image.shape[2] < 220:

                print("Before", image.shape, mask.shape)
                diff = 220 - image.shape[2]
                print("Diff", diff, case)
                
                image = np.concatenate([image] + [image[:, :, -1][:, :, np.newaxis]] * diff, axis=2)
                mask = np.concatenate([mask] + [mask[:, :, -1][:, :, np.newaxis]] * diff, axis=2)

                print("after", image.shape, mask.shape)
                
        if image.shape[2] > 220:
                print("Before", image.shape, mask.shape)
                image = image[:, :, :220]
                mask = mask[:, :, :220]
                print("after", image.shape, mask.shape)

        affine = image_raw.affine

        images.append(image)
        labels.append(mask)
        affines.append(affine)
        cases.append(case)

Before (256, 256, 228) (256, 256, 228)
after (256, 256, 220) (256, 256, 220)
Before (256, 256, 221) (256, 256, 221)
after (256, 256, 220) (256, 256, 220)
Before (256, 256, 228) (256, 256, 228)
after (256, 256, 220) (256, 256, 220)
Before (256, 256, 218) (256, 256, 218)
Diff 2 A114
after (256, 256, 220) (256, 256, 220)
Before (256, 256, 228) (256, 256, 228)
after (256, 256, 220) (256, 256, 220)


In [3]:
hf = h5py.File("data.h5", "w")

hf.create_dataset("images", data=images)
hf.create_dataset("masks", data=labels)
hf.create_dataset("affines", data=affines)
hf.create_dataset("names", data=cases)

hf.close()



: 

: 